In [ ]:
from pathlib import Path
import requests
from tqdm import tqdm

phoible_data_url = "https://raw.githubusercontent.com/phoible/dev/v2.0/data/phoible.csv"
phobile_file_path = "./data/phoible.csv"

if not Path(phobile_file_path).exists():
    response = requests.get(phoible_data_url, stream=True)

    with open(phobile_file_path, "wb") as fh:
        for data in tqdm(response.iter_content()):
            fh.write(data)

In [ ]:
import pandas as pd
import numpy as np

phoible_df = pd.read_csv(phobile_file_path)
print(phoible_df.columns)

In [ ]:
consonants_plumonic_df = pd.read_csv("./consonants_plumonic.csv", dtype=str, index_col=[0], keep_default_na=False, na_values=["-1"])
consonants_plumonic_df

In [ ]:
consonants = set(consonants_plumonic_df.stack().replace("", np.nan).dropna().unique().tolist())

phobile_consonants_df = phoible_df[phoible_df["SegmentClass"] == "consonant"][["LanguageName", "SpecificDialect", "Phoneme"]]
phobile_consonants_df.fillna({"SpecificDialect": "none"}, inplace=True)

grouped = phobile_consonants_df.head(100).groupby(["LanguageName", "SpecificDialect"])

for grp, data in grouped:
    print(grp)
    lang_consonants = set(data.Phoneme.to_list()).intersection(consonants)
    result = np.where(consonants_plumonic_df.isin(lang_consonants), 1, 0)
    print(result)

# for c in consonants:
#     result = np.where(consonants_plumonic_df == c, 1, 0)
#     print(result)